In [32]:
import cv2

In [33]:
import dlib

In [34]:
import tensorflow as tf

In [35]:
from imutils.video import FPS

In [36]:
import numpy as np

In [37]:
import argparse

In [38]:
import os

In [39]:
subjects = ["", "Elvis Preseley"]

In [40]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    face_cascade = cv2.CascadeClassifier('xml/lbpcascade_frontalface.xml')
 
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
 
    if (len(faces) == 0):
        return None, None
 
    (x, y, w, h) = faces[0]
    return gray[y:y+w, x:x+h], faces[0]

In [41]:
def prepare_training_data(data_folder_path):
 
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)

    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []

    #let's go through each directory and read images within it
    for dir_name in dirs:
        if not dir_name.startswith("s"):
            continue;
        label = int(dir_name.replace("s", ""))

    
    subject_dir_path = data_folder_path + "/" + dir_name

    #get the images names that are inside the given subject directory
    subject_images_names = os.listdir(subject_dir_path)

    #------STEP-3--------
    #go through each image name, read image, 
    #detect face and add face to list of faces
    for image_name in subject_images_names:

    #ignore system files like .DS_Store
        if image_name.startswith("."):
            continue;

    #build image path
    #sample image path = training-data/s1/1.pgm
        image_path = subject_dir_path + "/" + image_name

    #read image
        image = cv2.imread(image_path)

    #display an image window to show the image 
        cv2.imshow("Training on image...", image)
        cv2.waitKey(100)

    #detect face
        face, rect = detect_face(image)

    #------STEP-4--------
    #for the purpose of this tutorial
    #we will ignore faces that are not detected
        if face is not None:
        #add face to list of faces
            faces.append(face)
        #add label for this face
            labels.append(label)

    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()

    return faces, labels

In [42]:
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  6
Total labels:  6


In [43]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

In [44]:
face_recognizer.train(faces, np.array(labels))

In [45]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [52]:
def predict(test_img):
#make a copy of the image as we don't want to change original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label = face_recognizer.predict(face)
    label = np.array(label)


    #get name of respective label returned by face recognizer
    label_text = subjects[label]
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
   # np.array(label)
    return img

In [51]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread("images/test1.jpg")
test_img2 = cv2.imread("images/test2.jpg")

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], predicted_img1)
cv2.imshow(subjects[2], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...


TypeError: list indices must be integers or slices, not tuple